In [ ]:
### extracting locations (non-Google) from initial.ipynb
### superceded by calls to google maps API 
import pandas as pd
from geopy.geocoders import Nominatim
from time import sleep

# Load locations from a CSV file
locs = pd.DataFrame(df.Location.unique(), columns=["location"])

geolocator = Nominatim(user_agent="geo_lookup")

# Function to get latitude and longitude
def get_coordinates(location):
    try:
        loc = geolocator.geocode(location, timeout=10)
        if loc:
            return pd.Series([loc.latitude, loc.longitude])
        else:
            return pd.Series([None, None])
    except Exception as e:
        print(f"Error for {location}: {e}")
        return pd.Series([None, None])

# Apply function with a delay to avoid API rate limits
locs[["latitude", "longitude"]] = locs["location"].apply(lambda x: get_coordinates(x))


In [ ]:
### automatic clustering from analysis.ipynb
### superceded by tolerance-based clustering


# test_doi = "10.3354/meps11591"
test_doi = carb_df.doi.unique()[1]
test_df = carb_df[carb_df.doi == test_doi].copy()

# assign species_types values to unique ints. This is handled later by grouping, but this is useful for a quick demo
# species_types = test_df["species_types"].unique()
# species_types_dict = {species_type: i for i, species_type in enumerate(species_types)}
# test_df.loc[:,"species_types"] = test_df["species_types"].map(species_types_dict)
# select control variables
# treatment_data = test_df[['t_in', 'phtot', 'irr', 'species_types']].dropna()
treatment_data = test_df[['t_in', 'phtot', 'irr']].dropna()
# determine optimal number of clusters in the data
optimal_k, score_list = utils.optimal_kmeans(treatment_data)
# fit KMeans with the optimal number of clusters
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
test_df.loc[:,'treatment_group'] = kmeans.fit_predict(treatment_data)

# # visualise clustering score wrt number of clusters
# plt.figure(figsize=(4,4))
# plt.scatter(*zip(*score_list), marker='o', linestyle='-')
# plt.xlabel('Number of Clusters')
# plt.ylabel('Silhouette Score (higher is better)')
# # mark a vertical line at the optimal number of clusters
# plt.axvline(optimal_k, color='r', linestyle='--', label='Optimal # of clusters')
# plt.legend()
# plt.show()